# Kyriba X Imperial College  Hackathon 


In [ ]:
#@title Import modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import datetime
import sys, os
from pathlib import Path
import json
from forex_python.converter import CurrencyCodes
import pgeocode

In [ ]:
seed=0
np.random.seed(seed)

Some libraries have to be installed : 

In [ ]:
! pip install forex-python

You should consider upgrading via the 'C:\Users\basile.etienne\Documents\env\Scripts\python.exe -m pip install --upgrade pip' command.


In [ ]:
! pip install pgeocode

In [ ]:
nomi = pgeocode.Nominatim('FR')
nomi.query_postal_code("62140")

postal_code                                                   62140
country_code                                                     FR
place_name        Wambercourt, Bouin-Plumoison, Caumont, Marconn...
state_name                                          Hauts-de-France
state_code                                                     32.0
county_name                                           Pas-de-Calais
county_code                                                      62
community_name                          Arrondissement de Montreuil
community_code                                                  624
latitude                                                    50.4284
longitude                                                    2.0136
accuracy                                                          5
Name: 0, dtype: object

The following function transforms the input json from the ERP to a more convenient pandas DataFrame.

In [ ]:
def json_to_df(path) :
    # Opening JSON file
    f = open(path, encoding="utf8")

    # returns JSON object as
    # a dictionary
    data = json.load(f)
    dicts = data['body']['value']
    df = pd.DataFrame(dicts[0], index= [0])
    for k in range(len(dicts)) :
        row = pd.DataFrame(dicts[k], index= [k])
        df = df.append(row, ignore_index=True)
    return df

## e.g. with a json from D365
df_D365 = json_to_df('D365.json')

In [ ]:
df_D365.columns

Index(['@odata.etag', 'ItemInternalId', 'KeyGuid', 'Correspondent_Account',
       'ThirdPartyBank_ProvinceCode', 'Mandate_MandateDate',
       'ThirdPartyBank_CountryCode', 'ThirdPartyBank_Description', 'LinkId',
       'Invoice_InvoiceDate', 'CheckNumber', 'PaymentStatus', 'FreeText8',
       'FreeText9', 'TransactionAmount', 'FreeText2', 'FreeText3',
       'ThirdPartyBank_Address2', 'FreeText1', 'ThirdPartyBank_Address1',
       'Details', 'FeeAssessment', 'FreeText6', 'FreeText7', 'FreeText4',
       'FreeText5', 'ThirdParty_ProvinceCode', 'ThirdParty_Description',
       'ThirdPartyBank_ZIP', 'Correspondent_CountryCode', 'TransactionDate',
       'ThirdPartyBank_AccountId1', 'CompanyId', 'Correspondent_Description',
       'TransactionType', 'ThirdParty_City', 'Correspondent_BIC',
       'ThirdPartyBank_City', 'Invoice_PaidAmount',
       'ThirdPartyBank_IdentifierValue', 'Invoice_Type',
       'ThirdParty_Description2', 'Reference', 'Reason1', 'Account',
       'ThirdParty_Ident

## Columns from Kyriba
These are the output columns you need to provide. 

In [ ]:
df = pd.read_csv('cols_names.csv') 

Respectively the columns for transactions and for invoices :

In [ ]:
kyr_cols_TR = df[df['Position in CSV Kyriba file'].notnull() & df['Position in CSV Kyriba file'].str.contains('TR', case = False)]['IM column']
kyr_cols_TR = kyr_cols_TR[kyr_cols_TR.notnull()].to_numpy()

kyr_cols_IN = df[df['Position in CSV Kyriba file'].notnull() & df['Position in CSV Kyriba file'].str.contains('IN', case = False)]['IM column']
kyr_cols_IN = kyr_cols_IN[kyr_cols_IN.notnull()].to_numpy()

**The output of your Model must be a pandas dataframe with `kyr_cols_TR` and `kyr_cols_IN` as columns respectively for the transactions and invoices.**

In [ ]:
print("###################### Columns for transactions : #####################")
for c in kyr_cols_TR : 
      print(c)
print("###################### Columns for invoices : ######################")

for c in kyr_cols_IN :
    print(c)
      

###################### Columns for transactions : #####################
Record Type
Transaction code
Reference
Debit/Credit account
Transaction currency
Transaction amount
Date
Reason 1
Fee assessment
Third party.Description
Third party.Description 2
Third party.Identifier value
Third party.Corp Id code
Third party.Address 1
Third party.Address 2
Third party.City
Third party.ZIP
Third party.Province code
Third party.Country code
Third party Contact.E-mail
Third party bank.Description
Third party bank.BIC Code
Third party bank.Identifier value
Third party bank.Address 1
Third party bank.Address 2
Third party bank.City
Third party bank.ZIP
Third party bank.Province code
Third party bank.Country code
Third party bank.Currency
Third party bank.Account Id 1
Third party bank.Account nature
Correspondent.BIC
Correspondent.Description
Correspondent.Country code
Correspondent.Identifier value
Correspondent.Account
Free text 1
Free text 2
Free text 3
Free text 4
Free text 5
Free text 6
Free text

## Rules
To automatically verify that fields are correctly mapped, a first set of rules has to be defined - and can be improved, before sending the TR/IN to the API. 

A date format must be specified, which could be learned, or specified manually. See https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html and https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior for the formats. For more complex formats, see https://lexpredict-lexnlp.readthedocs.io/en/latest/modules/extract/en/dates.html#extract-en-dates for example. An example is provided by default.

In [ ]:
def respects_rules(df, date_format='%d/%m/%Y', record_type = 'TR') :
    df = df.copy()
    errors = [{} for k in range(len(df))]
    if record_type == 'TR' :
        for k in range(len(df)) :
            row = df.loc[k]
            #Type : IN / TR ?
            if row['Record Type'] != 'TR' :
                errors[k]['Record Type'] = 'Wrong record type, expected TR but got : ' + row['Record Type']
            
            # Currency
            if row['Transaction currency'] != '' :
                c = CurrencyCodes()
                if c.get_symbol(row['Transaction currency']) == None and c.get_currency_name(row['Transaction currency']) == None and c.get_currency_code_from_symbol(row['Transaction currency']) == None :
                    errors[k]['Transaction currency'] = 'Wrong value, expected a currency name, symbol or code, but got : ' + row['Transaction currency']
            # Date
            try :
                row.date = pd.to_datetime(row['Date'], format=date_format)
            except ValueError :
                errors[k]['Date Format'] = "Time data does not match format " + date_format
                
            #Bank account nature ''
            if row['Third party bank.Account nature'] not in ["01","02"] :
                errors[k]['Bank account nature'] = 'Wrong bank account nature, expected "01" or "02" but got : ' + row['Third party bank.Account nature']
            
            #Country, postal code, city
            if row['Third party bank.Country code'] != '' :
                try : 
                    nomi = pgeocode.Nominatim(row['Third party bank.Country code'])
                    if type(nomi.query_postal_code(row['Third party bank.ZIP'])['country_code']) == float :
                        errors[k]['ZIP'] = "Wrong ZIP/Postal code : " + row['Third party bank.ZIP'] + ", for the given country : " + row['Third party bank.Country code']
                    elif  row['Third party bank.City'] != '' and row['Third party bank.City'] not in nomi.query_postal_code(row['Third party bank.ZIP'])['place_name'].split(", ") :
                        errors[k]['City'] = "Wrong City : " + row['Third party bank.City'] + ", for the given ZIP/Postal code : " + row['Third party bank.ZIP'] + " and country " + row['Third party bank.Country code']

                except ValueError : 
                    errors[k]['Country code'] = "Wrong country code, either a not supported country or a wrong value. Got : " + row['Third party bank.Country code']
                
                
            if row['Third party.Country code'] != '' :
                try : 
                    nomi = pgeocode.Nominatim(row['Third party.Country code'])
                    if type(nomi.query_postal_code(row['Third party.ZIP'])['country_code']) == float :
                        errors[k]['ZIP'] = "Wrong ZIP/Postal code : " + row['Third party.ZIP'] + ", for the given country : " + row['Third party.Country code']
                    elif  row['Third party.City'] != '' and row['Third party.City'] not in nomi.query_postal_code(row['Third party.ZIP'])['place_name'].split(", ") :
                        errors[k]['City'] = "Wrong City : " + row['Third party.City'] + ", for the given ZIP/Postal code : " + row['Third party.ZIP'] + " and country " + row['Third party.Country code']

                except ValueError : 
                    errors[k]['Country code'] = "Wrong country code, either a not supported country or a wrong value. Got : " + row['Third party bank.Country code']
                
            if errors[k] != {} :
                errors[k]['index'] = k
                
            
         
    elif record_type == 'IN' : 
        for k in range(len(df)) :
            row = df.loc[k]
            try :
                row.due_date = pd.to_datetime(row['Due date'], format=date_format)
                row.invoice_date = pd.to_datetime(row['Invoice date'], format=date_format)
                if row.invoice_date > row.due_date :
                    errors[k]['Order of dates'] = 'The specified invoice date is greater than the due date'
            except ValueError :
                errors[k]['Date Format'] = "Time data does not match format " + date_format
            

            
            
            if errors[k] != {} :
                errors[k]['index'] = k
    return errors
    

In [ ]:
df_test =  pd.DataFrame([['TR', 'EUR','5/4/2020','02/4/2020', 'FR', '62140', '01','02/4/2020', 'Wambercourt'], 
                         ['TR', 'EUR','1/4/2020','02/4/2020', 'US', '22313', '01','02/4/2020', 'New York']], 
                        columns=['Record Type', 'Transaction currency', 'Due date', 'Invoice date', 'Third party bank.Country code', 
                                 'Third party bank.ZIP', 'Third party bank.Account nature','Date', 'Third party bank.City'], index=[0, 1])

In [ ]:
df_test

,Record Type,Transaction currency,Due date,Invoice date,Third party bank.Country code,Third party bank.ZIP,Third party bank.Account nature,Date,Third party bank.City
0,TR,EUR,5/4/2020,02/4/2020,FR,62140,01,02/4/2020,Wambercourt
1,TR,EUR,1/4/2020,02/4/2020,US,22313,01,02/4/2020,New York


In [ ]:
respects_rules(df_test, record_type = 'TR')

[{},
 {'City': 'Wrong City : New York, for the given ZIP/Postal code : 22313',
  'index': 1}]

The following function transforms such an input to the Kyriba format. Note that the input df should be only of one record type (either TR or IN) and the **order** of the columns **does matter**.

In [ ]:
#should we concatenate TR and corresponding IN ??
def to_kyr_format(df):
    res = []
    for k in range(len(df)) : 
        row = df.loc[k].to_numpy()
        string = ''
        for value in row :
            
            string += value + '|' #np.array2string(row.to_numpy(),separator='|')[1:-1]
        string = string[:-1]
        res.append(string)
    return res

In [ ]:
to_kyr_format(df_test)

['TR|EUR|5/4/2020|02/4/2020|FR|62140|01|02/4/2020',
 'TR|EUR|1/4/2020|02/4/2020|US|22313|01|02/4/2020']